# Test the functionality of DataPlane Python bindings

Mimic the `e2sar_reas_test` in the C++ unit test suite.

In [1]:
import time
import sys

## IMPORTANT: Update the path to your built Python module. Use the absolute path to make life easier.
sys.path.append(
    '/home/xmei/Documents/hpdf_projects/E2SAR/build/src/pybind')

import e2sar_py

In [2]:

# dir(e2sar_py.DataPlane)
# dir(e2sar_py.DataPlane.Reassembler)
# dir(e2sar_py.DataPlane.Segmenter)

In [3]:
DP_IPV4 = "127.0.0.1"
data_id = 0x0505   # decimal value: 1085
eventSrc_id = 0x11223344   # decimal value: 287454020

## DPReasTest1

This is a test that uses local host to send/receive fragments. It does NOT use control plane.


1. First initialize "Segmenter" objects.

In [4]:
# Set up URI for segmenter
SEG_URI = f"ejfat://useless@192.168.100.1:9876/lb/1?sync=192.168.0.1:12345&data={DP_IPV4}"
seg_uri = e2sar_py.EjfatURI(uri=SEG_URI, tt=e2sar_py.EjfatURI.TokenType.instance)

# Set up sflags
sflags = e2sar_py.DataPlane.SegmenterFlags()
sflags.useCP = False  # turn off CP. Default value is True
sflags.syncPeriodMs = 1000
sflags.syncPeriods = 5

assert(sflags.syncPeriodMs == 1000)
assert(sflags.useCP == False)
assert(sflags.syncPeriods == 5)

print("Segmenter flags:")
print(f"  syncPeriodMs={sflags.syncPeriodMs}")
print(f"  useCP={sflags.useCP}")
print(f"  mtu={sflags.mtu}")
print(f"  syncPeriods={sflags.syncPeriods}")

Segmenter flags:
  syncPeriodMs=1000
  useCP=False
  mtu=1500
  syncPeriods=5


In [5]:
# Init segmenter object
seg = e2sar_py.DataPlane.Segmenter(seg_uri, data_id, eventSrc_id, sflags)

The message we need to send.

In [6]:
SEND_STR = "THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS."

send_context = SEND_STR.encode('utf-8')
print(send_context)
print(len(send_context))

b'THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.'
66


2. Initilize the Reassembler.

In [7]:
# Set the reassembler URI
REAS_URI_ = f"ejfat://useless@192.168.100.1:9876/lb/1?sync=192.168.0.1:12345&data={DP_IPV4}"
reas_uri = e2sar_py.EjfatURI(uri=REAS_URI_, tt=e2sar_py.EjfatURI.TokenType.instance)

# Make sure the token matches the one in the string
def get_inst_token(uri : e2sar_py.EjfatURI):
    try:
        token = uri.get_instance_token().value()
        print("Instance Token:", token)
    except RuntimeError as e:
        print("Instance Token - Error:", e)

get_inst_token(reas_uri)

# Get data plane IPv4 address
print("Data Plane Address (v4):", str(reas_uri.get_data_addr_v4().value()[0]))

# Config the reassembler flags

rflags = e2sar_py.DataPlane.ReassemblerFlags()
rflags.useCP = False  # turn off CP. Default value is True
rflags.withLBHeader = True  # LB header will be attached since there is no LB

assert rflags.period_ms == 100  # default value of the C++ constructor
assert rflags.useCP == False

print("Reassembler flags:")
print(f"  period_ms={rflags.period_ms}")  # should be 100 according to the C++ constructor
print(f"  useCP={rflags.useCP}")
print(f"  validateCert = {rflags.validateCert}")
print(f"  epoch_ms = {rflags.epoch_ms}")
print(f"  setPoint = {rflags.setPoint}")
print(f"  [Kp, Ki, Kd] = [{rflags.Kp}, {rflags.Ki}, {rflags.Kd}]")
print(f"  dpV6 = {rflags.dpV6}")
print(f"  cpV6 = {rflags.cpV6}")
print(f"  portRange = {rflags.portRange}")
print(f"  withLBHeader = {rflags.withLBHeader}")

# Init the reassembler object
# print(type(reas_uri), type(1), type(rflags))
reas = e2sar_py.DataPlane.Reassembler(reas_uri, 1, rflags)


Instance Token: useless
Data Plane Address (v4): 127.0.0.1
Reassembler flags:
  period_ms=100
  useCP=False
  validateCert = True
  epoch_ms = 1000
  setPoint = 0.0
  [Kp, Ki, Kd] = [0.0, 0.0, 0.0]
  dpV6 = False
  cpV6 = False
  portRange = -1
  withLBHeader = True


In [8]:

res = reas.OpenAndStart()  # the DP address must be available
assert res.value() == 0

In [9]:
res = reas.getStats()
print(res)

(0, 0, 0, 0, 0, <E2SARErrorc.NoError: 0>)


3. Send the contexts with the Segmenter object.

In [10]:
# Start segmenter threads
res = seg.OpenAndStart()
assert res.value() == 0

res = seg.getSendStats()
if (res[1] != 0):
    print(f"Error encountered after opening send socket: {res[2]}")
    # exit(-1)

In [11]:
# Send the data
for i in range(5):
    print(f"Sending data...                Rd {i}")
    res = seg.addToSendQueue(send_context, len(send_context))
    assert(res.value() == 0)
    res = seg.getSendStats()
    if (res[1] != 0):
        print(f"  SendStats: {res}")
        print(f"  Error encountered sending event frame: {i}, error: {res[2]}")
    time.sleep(1)

res = seg.getSendStats()
# assert(res[0] == 5)  # hold for the first run
print(f"Sent {res[0]} data frames")

if (res[1] != 0):
    print(f"SendStats: {res}")
    print(f"After sending data --  error: {res[2]}")
    # exit()

Sending data...                Rd 0
Sending data...                Rd 1
Sending data...                Rd 2
Sending data...                Rd 3
Sending data...                Rd 4
Sent 5 data frames


In [12]:

# Prepare Python list to hold the output data
recv_bytes_list = [None]  # Placeholder for the byte buffer

# Call getEvent on the instance
for i in range(5):
    print(f"Receiving data...                Rd {i}")
    res, recv_event_len, recv_event_num, recv_data_id = reas.getEvent(recv_bytes_list)

    if (res != 0):  # -1 or -2 (has_error)
        continue
    recv = recv_bytes_list[0].decode('utf-8')
    print(f"  recv_buf:\t {recv}")
    assert(recv_data_id == data_id)
    print(f"  bufLen:\t {recv_event_len}")
    print(f"  eventNum:\t {recv_event_num}")
    print(f"  dataId:\t {recv_data_id}")


Receiving data...                Rd 0
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 0
  dataId:	 1285
Receiving data...                Rd 1
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 1
  dataId:	 1285
Receiving data...                Rd 2
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 2
  dataId:	 1285
Receiving data...                Rd 3
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 3
  dataId:	 1285
Receiving data...                Rd 4
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 4
  dataId:	 1285


## DPReasTest2

Test segmentation and reassembly on local host with no control plane (basic segmentation) using small MTU.

In [13]:
# Set up sflags
sflags2 = e2sar_py.DataPlane.SegmenterFlags()
sflags2.useCP = False  # turn off CP. Default value is True
sflags2.syncPeriodMs = 1000
sflags2.syncPeriods = 5
sflags2.mtu = 80

assert(sflags2.syncPeriodMs == 1000)
assert(sflags2.useCP == False)
assert(sflags2.syncPeriods == 5)
assert(sflags2.mtu == 80)

print("Segmenter flags:")
print(f"  syncPeriodMs={sflags2.syncPeriodMs}")
print(f"  useCP={sflags2.useCP}")
print(f"  mtu={sflags2.mtu}")
print(f"  syncPeriods={sflags2.syncPeriods}")

Segmenter flags:
  syncPeriodMs=1000
  useCP=False
  mtu=80
  syncPeriods=5


In [14]:
SEG_URI2 = f"ejfat://useless@192.168.100.1:9876/lb/1?sync=192.168.0.1:12345&data={DP_IPV4}"
seg_uri2 = e2sar_py.EjfatURI(uri=SEG_URI2, tt=e2sar_py.EjfatURI.TokenType.instance)

In [15]:
# Initialize a new Segmenter objects
seg2 = e2sar_py.DataPlane.Segmenter(seg_uri2, data_id, eventSrc_id, sflags2)

res = seg2.OpenAndStart()
assert(res.value() == 0)

res = seg2.getSendStats()
if (res[1] != 0):
    print(f"Error encountered after opening send socket: {res[2]}")
    # exit(-1)

In [16]:
# Send data with the small MTU
for i in range(5):
    print(f"Sending data...                Rd {i}")
    res = seg2.addToSendQueue(send_context, len(send_context))
    assert(res.value() == 0)
    res = seg2.getSendStats()
    # print(f"  getSendStats:\t {res}")
    if (res[1] != 0):
        print(f"  SendStats: {res}")
        print(f"  Error encountered sending event frame: {i}, error: {res[2]}")
    time.sleep(1)

res = seg2.getSendStats()
# assert(res[0] == 25)  # hold for the 1st run
print(f"Sent {res[0]} data frames")

if (res[1] != 0):
    print(f"SendStats: {res}")
    print(f"After sending data --  error: {res[2]}")
    # exit()

Sending data...                Rd 0
Sending data...                Rd 1
Sending data...                Rd 2
Sending data...                Rd 3
Sending data...                Rd 4
Sent 25 data frames


In [17]:
# Prepare Python list to hold the output data
recv_bytes_list = [None]  # Placeholder for the byte buffer

# Call getEvent on the instance
for i in range(5):
    print(f"Receiving data...                Rd {i}")
    res, recv_event_len, recv_event_num, recv_data_id = reas.getEvent(recv_bytes_list)

    if (res != 0):  # -1 or -2 (has_error)
        continue
    recv = recv_bytes_list[0].decode('utf-8')
    print(f"  recv_buf:\t {recv}")
    assert(recv_data_id == data_id)
    print(f"  bufLen:\t {recv_event_len}")
    print(f"  eventNum:\t {recv_event_num}")
    print(f"  dataId:\t {recv_data_id}")

Receiving data...                Rd 0
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 0
  dataId:	 1285
Receiving data...                Rd 1
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 1
  dataId:	 1285
Receiving data...                Rd 2
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 2
  dataId:	 1285
Receiving data...                Rd 3
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 3
  dataId:	 1285
Receiving data...                Rd 4
  recv_buf:	 THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.
  bufLen:	 66
  eventNum:	 4
  dataId:	 1285


## DPReasTest3

Test creationg of reassemblers with different parameters. No real sending or receiving code is executed.

In [18]:
# Create reassembler with 1 recv thread

rflags_m = e2sar_py.DataPlane.ReassemblerFlags()
reas_a = e2sar_py.DataPlane.Reassembler(reas_uri, 1, rflags_m)

print("This reassembler has")
print(f"   {reas_a.get_numRecvThreads()} threads;")
print(f"   listening on ports {reas_a.get_recvPorts()[0]}:{reas_a.get_recvPorts()[1]};")
print(f"   using portRange: {reas_a.get_portRange()}.")

assert reas_a.get_numRecvThreads() == 1
assert reas_a.get_recvPorts() == (19522, 19522)
assert reas_a.get_portRange() == 0

This reassembler has
   1 threads;
   listening on ports 19522:19522;
   using portRange: 0.


In [19]:
# Create reassembler with 4 recv threads
reas_b = e2sar_py.DataPlane.Reassembler(reas_uri, 4, rflags_m)

print("This reassembler has")
print(f"   {reas_b.get_numRecvThreads()} threads;")
print(f"   listening on ports {reas_b.get_recvPorts()[0]}:{reas_b.get_recvPorts()[1]};")
print(f"   using portRange: {reas_b.get_portRange()}.")

assert reas_b.get_numRecvThreads() == 4
assert reas_b.get_recvPorts() == (19522, 19525)
assert reas_b.get_portRange() == 2

This reassembler has
   4 threads;
   listening on ports 19522:19525;
   using portRange: 2.


In [20]:
# Create reassembler with 7 recv threads
reas_c = e2sar_py.DataPlane.Reassembler(reas_uri, 7, rflags_m)

print("This reassembler has")
print(f"   {reas_c.get_numRecvThreads()} threads;")
print(f"   listening on ports {reas_c.get_recvPorts()[0]}:{reas_c.get_recvPorts()[1]};")
print(f"   using portRange: {reas_c.get_portRange()}.")

assert reas_c.get_numRecvThreads() == 7
assert reas_c.get_recvPorts() == (19522, 19529)
assert reas_c.get_portRange() == 3

This reassembler has
   7 threads;
   listening on ports 19522:19529;
   using portRange: 3.


In [21]:
# 4 threads with portRange override
rflags_m.portRange = 10

reas_d = e2sar_py.DataPlane.Reassembler(reas_uri, 4, rflags_m)

print("This reassembler has")
print(f"   {reas_d.get_numRecvThreads()} threads;")
print(f"   listening on ports {reas_d.get_recvPorts()[0]}:{reas_d.get_recvPorts()[1]};")
print(f"   using portRange: {reas_d.get_portRange()}.")

assert reas_d.get_numRecvThreads() == 4
assert reas_d.get_recvPorts() == (19522, 20545)
assert reas_d.get_portRange() == 10


This reassembler has
   4 threads;
   listening on ports 19522:20545;
   using portRange: 10.


In [22]:
# 4 threads with low portRange override
rflags_m.portRange = 1

reas_e = e2sar_py.DataPlane.Reassembler(reas_uri, 4, rflags_m)

print("This reassembler has")
print(f"   {reas_e.get_numRecvThreads()} threads;")
print(f"   listening on ports {reas_e.get_recvPorts()[0]}:{reas_e.get_recvPorts()[1]};")
print(f"   using portRange: {reas_e.get_portRange()}.")

assert reas_e.get_numRecvThreads() == 4
assert reas_e.get_recvPorts() == (19522, 19523)
assert reas_e.get_portRange() == 1

This reassembler has
   4 threads;
   listening on ports 19522:19523;
   using portRange: 1.
